<a href="https://www.kaggle.com/code/sushovansaha9/bengaligemma-finetune-gemma3-bengalidata-unsloth?scriptVersionId=288661641" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

![image.png](https://raw.githubusercontent.com/ambideXtrous9/BengaliGemma/refs/heads/main/gemmaUnsloth.png)

## **🦥 Install Packages**

In [1]:
!nvidia-smi

Sat Dec 27 07:15:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install -qq -U evaluate rouge_score

In [3]:
from transformers import set_seed

SEED = 42
set_seed(SEED)


2025-12-27 07:16:01.617835: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766819761.804315      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766819761.853310      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766819762.290881      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766819762.290913      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766819762.290916      55 computation_placer.cc:177] computation placer alr

In [4]:
import unsloth
from unsloth import FastLanguageModel
import torch

/tmp/ipykernel_55/1012355901.py:1: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
import os
os.environ["UNSLOTH_RETURN_LOGITS"] = "1"

In [6]:
max_seq_length = 1024

## **🦥 Loading Gemma3 Model**

In [7]:
# Load the base model and tokenizer

model_name = "unsloth/gemma-3-1b-it-unsloth-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,   # Define context length
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # token = "hf_...",      # Add your token if using a gated model
)

==((====))==  Unsloth 2025.12.9: Fast Gemma3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [8]:
# Add LoRA adapters to the model
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,           # LoRA rank (higher rank = more parameters, potentially better fit but more memory)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", # Target attention and MLP layers
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,  # Scaling factor (often set to r or 2*r)
    lora_dropout = 0, # Dropout probability for LoRA layers
    bias = "none",    # Fine-tuning bias terms ('none' is often optimal)
    # Use Unsloth's gradient checkpointing for memory saving
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False, # Rank Stable LoRA (optional)
    loftq_config = None, # LoftQ initialization (optional)
)

Unsloth: Making `model.base_model.model.model` require gradients


In [9]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [10]:
print(tokenizer.chat_template)

{{ bos_token }}
{%- if messages[0]['role'] == 'system' -%}
    {%- if messages[0]['content'] is string -%}
        {%- set first_user_prefix = messages[0]['content'] + '

' -%}
    {%- else -%}
        {%- set first_user_prefix = messages[0]['content'][0]['text'] + '

' -%}
    {%- endif -%}
    {%- set loop_messages = messages[1:] -%}
{%- else -%}
    {%- set first_user_prefix = "" -%}
    {%- set loop_messages = messages -%}
{%- endif -%}
{%- for message in loop_messages -%}
    {%- if (message['role'] == 'user') != (loop.index0 % 2 == 0) -%}
        {{ raise_exception("Conversation roles must alternate user/assistant/user/assistant/...") }}
    {%- endif -%}
    {%- if (message['role'] == 'assistant') -%}
        {%- set role = "model" -%}
    {%- else -%}
        {%- set role = message['role'] -%}
    {%- endif -%}
    {{ '<start_of_turn>' + role + '
' + (first_user_prefix if loop.first else "") }}
    {%- if message['content'] is string -%}
        {{ message['content'] | trim }}


## **🦥 Loading BengaliChat Data**

### Non-Reasoning Dataset

In [11]:
from datasets import load_dataset

bengali_dataset = load_dataset("rishiraj/bengalichat")


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a3cbd3fa0a5ef2(…):   0%|          | 0.00/26.3M [00:00<?, ?B/s]

data/test-00000-of-00001-3b2e30253805f2b(…):   0%|          | 0.00/1.42M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

## **🦥 Loading Bengali Reasoning Data**

### Reasoning Dataset

In [12]:
beng_reason_data = load_dataset("KillerShoaib/DeepSeek-r1-Distill-Bangla-MMLU-Reasoning-Data")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/39.4M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/5.46M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17796 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2565 [00:00<?, ? examples/s]

In [13]:
nonreasontrain = bengali_dataset["train"]
nonreasontest = bengali_dataset["test"]

In [14]:
reasontrain = beng_reason_data["train"]
reasontest = beng_reason_data["test"]

In [15]:
print("Reason Train Data Length = ",len(reasontrain))
print("Reason Test Data Length = ",len(reasontest))
print("Non-Reason Train Data Length = ",len(nonreasontrain))
print("Non-Reason Test Data Length = ",len(nonreasontest))

Reason Train Data Length =  17796
Reason Test Data Length =  2565
Non-Reason Train Data Length =  9500
Non-Reason Test Data Length =  500


In [16]:
nonreasontest[100]

{'prompt': 'What are some Starbucks drinks I can enjoy if I’m lactose intolerant?',
 'prompt_id': '40995aedb42a2ce8a74fdc801dc7a3c7a4e50206d0820dfaa35b47f5bb76bf9a',
 'messages': [{'content': '', 'role': 'system'},
  {'content': 'আমি যদি ল্যাকটোজ অসহিষ্ণু হয়ে থাকি তবে আমি কিছু স্টারবাক্স পানীয় উপভোগ করতে পারি?',
   'role': 'user'},
  {'content': '- গোলাপী পানীয়: মসৃণ এবং সতেজকর, এই পানীয়টিতে স্ট্রবেরি এবং অ্যাকাই বেরি, নারকেল দুধ এবং আবেগের ফল রয়েছে।\n- মোচা ফ্রেপ্পুকিনো: নারকেল দুধ এবং বরফের চকোলেট মিশ্রণ।কোনও চাবুকের জন্য জিজ্ঞাসা করার বিষয়টি নিশ্চিত করুন!\n- আইসড কুমড়ো মশলা ল্যাট: পেটে ব্যথা ছাড়াই একটু মশালার মুডে?এই কুমড়ো, দারুচিনি, জায়ফল এবং লবঙ্গটি নারকেল দুধের সাথে পান করুন এবং কোনও চাবুক নেই!এছাড়াও, গরম চেষ্টা করুন!',
   'role': 'assistant'}],
 'category': 'Generation',
 'text': '<|system|>\n</s>\n<|user|>\nআমি যদি ল্যাকটোজ অসহিষ্ণু হয়ে থাকি তবে আমি কিছু স্টারবাক্স পানীয় উপভোগ করতে পারি?</s>\n<|assistant|>\n- গোলাপী পানীয়: মসৃণ এবং সতেজকর, এই পানীয়টিতে স্ট্রবেরি 

In [17]:
reasontest[100]

{'id': '0040288e-8580-4c00-b188-5f311cb38dcf-51896',
 'reasoning_translation': '<think>আচ্ছা, দেখা যাক। প্রশ্নটি হলো Tidal Volume এবং এর মান সম্পর্কে। আমার মনে আছে যে tidal volume হলো স্বাভাবিক শ্বাস-প্রশ্বাসের সময় ফুসফুসের ভিতরে এবং বাইরে যাওয়া বাতাসের পরিমাণ। কিন্তু অপেক্ষা করুন, সঠিক মানটি কত?\n\nহুমম, আমার মনে হয় শরীরবিদ্যায়, একজন সুস্থ প্রাপ্তবয়স্কের জন্য গড় tidal volume প্রায় 500 মিলিলিটার। তবে কখনও কখনও লোকেরা এটিকে অন্যান্য ভলিউমের সাথে বিভ্রান্ত করতে পারে যেমন inspiratory reserve volume বা expiratory reserve volume। আমাকে নিশ্চিত হতে দিন।\n\nঅপেক্ষা করুন, অপশনগুলো হলো A:500 ml, B:400, C:200, D:300। সুতরাং স্ট্যান্ডার্ড উত্তরটি 500 ml হওয়া উচিত। তবে এখানে কোনও কৌশল আছে কি? যেমন, বিশ্রামের সময় নাকি ব্যায়ামের সময় এটি আলাদা? না, tidal volume বিশেষভাবে স্বাভাবিক, বিশ্রামের শ্বাস-প্রশ্বাসের সময় হয়ে থাকে। সুতরাং হ্যাঁ, 500 ml। সুতরাং সঠিক উত্তরটি A হওয়া উচিত।</think>',
 'ans_translation': 'Tidal Volume হলো স্বাভাবিক শ্বাস-প্রশ্বাসের সময় ফুসফুসে প্রবেশ ও নির্গত হওয়া বাতাসে

## **🦥 Dataset Preprocessing**

In [18]:
def convert_nonreason_minimal_infini_format(examples):
    conversations = []
    sources = []
    scores = []

    for msgs in examples["messages"]:
        ua = [m for m in msgs if m["role"] in ("user", "assistant")]

        if len(ua) == 2 and ua[0]["role"] == "user":
            conversations.append([
                {"role": "user", "content": ua[0]["content"]},
                {"role": "assistant", "content": ua[1]["content"]},
            ])
            sources.append("bengalichat-openqa")
            scores.append(1.0)
        else:
            conversations.append(None)
            sources.append(None)
            scores.append(None)

    return {
        "conversations": conversations,
        "source": sources,
        "score": scores,
    }


In [19]:
def convert_reason_minimal_infini_format(examples):
    conversations = []
    sources = []
    scores = []

    n = len(examples["question"])

    for i in range(n):
        try:
            question = examples["question"][i]
            options = examples["options"][i]
            reasoning = examples["reasoning_translation"][i]
            answer = examples["ans_translation"][i]

            # ---- Validation ----
            if (
                question is None
                or options is None
                or reasoning is None
                or answer is None
                or not isinstance(options, (list, tuple))
                or len(options) < 4
            ):
                raise ValueError("Invalid sample")

            user_content = (
                f"প্রশ্ন: {question}\n\n"
                f"অপশন:\n"
                f"A. {options[0]}\n"
                f"B. {options[1]}\n"
                f"C. {options[2]}\n"
                f"D. {options[3]}"
            )

            assistant_content = f"{reasoning}\n\n{answer}"

            conversations.append([
                {"role": "user", "content": user_content},
                {"role": "assistant", "content": assistant_content},
            ])

            sources.append("bengalichat-openqa")
            scores.append(1.0)

        except Exception:
            # Mark bad rows for filtering
            conversations.append(None)
            sources.append(None)
            scores.append(None)

    return {
        "conversations": conversations,
        "source": sources,
        "score": scores,
    }


In [20]:
print("Reason Train Data Length = ",len(reasontrain))
print("Reason Test Data Length = ",len(reasontest))
print("Non-Reason Train Data Length = ",len(nonreasontrain))
print("Non-Reason Test Data Length = ",len(nonreasontest))

Reason Train Data Length =  17796
Reason Test Data Length =  2565
Non-Reason Train Data Length =  9500
Non-Reason Test Data Length =  500


### Non-Reason Train-Test Data

In [21]:
nonreason_train_dataset = nonreasontrain.map(
    convert_nonreason_minimal_infini_format,
    batched=True,
    remove_columns=nonreasontrain.column_names
)

nonreason_train_dataset = nonreason_train_dataset.filter(lambda x: x["conversations"] is not None)


Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9500 [00:00<?, ? examples/s]

In [22]:
nonreason_test_dataset = nonreasontest.map(
    convert_nonreason_minimal_infini_format,
    batched=True,
    remove_columns=nonreasontest.column_names
)

nonreason_test_dataset = nonreason_test_dataset.filter(lambda x: x["conversations"] is not None)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

### Reason Train-Test Data

In [23]:
reason_train_dataset = reasontrain.map(
    convert_reason_minimal_infini_format,
    batched=True,
    remove_columns=reasontrain.column_names
)

reason_train_dataset = reason_train_dataset.filter(lambda x: x["conversations"] is not None)


Map:   0%|          | 0/17796 [00:00<?, ? examples/s]

Filter:   0%|          | 0/17796 [00:00<?, ? examples/s]

In [24]:
reason_test_dataset = reasontest.map(
    convert_reason_minimal_infini_format,
    batched=True,
    remove_columns=reasontest.column_names
)

reason_test_dataset = reason_test_dataset.filter(lambda x: x["conversations"] is not None)


Map:   0%|          | 0/2565 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2565 [00:00<?, ? examples/s]

In [25]:
nonreason_test_dataset[100]

{'conversations': [{'content': 'আপনি যদি দুধের জন্য মুদি দোকানে ভ্রমণে যান এমন কোনও মহিলার সম্পর্কে একটি ছোট গল্প লিখেন তবে আমি এটি চাই।দয়া করে 200 টিরও কম শব্দ তৈরি করুন এবং শেষে একটি চমক তৈরি করুন।',
   'role': 'user'},
  {'content': 'মারিয়া রেফ্রিজারেটরে তাকিয়ে দীর্ঘশ্বাস ফেলল যখন সে দেখল যে পরিবারটি আবার দুধের বাইরে চলে গেছে।তিনি জানতেন যে তার বাচ্চাদের সকালে প্রাতঃরাশের জন্য দুধের প্রয়োজন ছিল এবং যদি তিনি এখনই দোকানে না যান তবে এটি সবার জন্য একটি বড় সমস্যা সৃষ্টি করবে।তিনি বাড়ির চারপাশে তাকালেন এবং দরজাটি বের করার আগে তার চাবিগুলি এবং তার মানিব্যাগটি পেয়েছিলেন।\n\nমারিয়া যখন স্টোরের সামনের দিকে টানল তখন সে দেখে হতবাক হয়ে গেল যে লাইটগুলি চালু আছে তবে কেউ ভবনে নেই।তিনি নির্জন আইলগুলির মধ্য দিয়ে সাবধানে হাঁটলেন, সবাই কোথায় গিয়েছিল সে সম্পর্কে ক্লু খুঁজছিলেন।হঠাৎ করেই, তিনি তার পিছনে আন্দোলন শুনেছিলেন।তিনি দ্রুত ঘুরে দাঁড়ালেন, লড়াইয়ের জন্য প্রস্তুত।\n\nতবে তার পিছনে কোনও ব্যক্তিকে খুঁজে পাওয়ার পরিবর্তে তিনি দেখলেন সিরিয়াল আইলের মাঝখানে একটি বড় দুধ গরু দাঁড়িয়ে আছে।গ

In [26]:
reason_test_dataset[100]

{'conversations': [{'content': 'প্রশ্ন: Tidal Volume এর মান কত?\n\nঅপশন:\nA. 500 ml\nB. 400 mL\nC. 200 mL\nD. 300 mL',
   'role': 'user'},
  {'content': '<think>আচ্ছা, দেখা যাক। প্রশ্নটি হলো Tidal Volume এবং এর মান সম্পর্কে। আমার মনে আছে যে tidal volume হলো স্বাভাবিক শ্বাস-প্রশ্বাসের সময় ফুসফুসের ভিতরে এবং বাইরে যাওয়া বাতাসের পরিমাণ। কিন্তু অপেক্ষা করুন, সঠিক মানটি কত?\n\nহুমম, আমার মনে হয় শরীরবিদ্যায়, একজন সুস্থ প্রাপ্তবয়স্কের জন্য গড় tidal volume প্রায় 500 মিলিলিটার। তবে কখনও কখনও লোকেরা এটিকে অন্যান্য ভলিউমের সাথে বিভ্রান্ত করতে পারে যেমন inspiratory reserve volume বা expiratory reserve volume। আমাকে নিশ্চিত হতে দিন।\n\nঅপেক্ষা করুন, অপশনগুলো হলো A:500 ml, B:400, C:200, D:300। সুতরাং স্ট্যান্ডার্ড উত্তরটি 500 ml হওয়া উচিত। তবে এখানে কোনও কৌশল আছে কি? যেমন, বিশ্রামের সময় নাকি ব্যায়ামের সময় এটি আলাদা? না, tidal volume বিশেষভাবে স্বাভাবিক, বিশ্রামের শ্বাস-প্রশ্বাসের সময় হয়ে থাকে। সুতরাং হ্যাঁ, 500 ml। সুতরাং সঠিক উত্তরটি A হওয়া উচিত।</think>\n\nTidal Volume হলো স্বাভাবিক শ্বা

In [27]:
from unsloth.chat_templates import standardize_data_formats

In [28]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

In [29]:
nonreason_train_dataset = standardize_data_formats(nonreason_train_dataset)
nonreason_test_dataset = standardize_data_formats(nonreason_test_dataset)
reason_train_dataset = standardize_data_formats(reason_train_dataset)
reason_test_dataset = standardize_data_formats(reason_test_dataset)

Unsloth: Standardizing formats (num_proc=4):   0%|          | 0/8705 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=4):   0%|          | 0/446 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=4):   0%|          | 0/17791 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=4):   0%|          | 0/2564 [00:00<?, ? examples/s]

In [30]:
nonreason_test_dataset[100]

{'conversations': [{'content': 'আপনি যদি দুধের জন্য মুদি দোকানে ভ্রমণে যান এমন কোনও মহিলার সম্পর্কে একটি ছোট গল্প লিখেন তবে আমি এটি চাই।দয়া করে 200 টিরও কম শব্দ তৈরি করুন এবং শেষে একটি চমক তৈরি করুন।',
   'role': 'user'},
  {'content': 'মারিয়া রেফ্রিজারেটরে তাকিয়ে দীর্ঘশ্বাস ফেলল যখন সে দেখল যে পরিবারটি আবার দুধের বাইরে চলে গেছে।তিনি জানতেন যে তার বাচ্চাদের সকালে প্রাতঃরাশের জন্য দুধের প্রয়োজন ছিল এবং যদি তিনি এখনই দোকানে না যান তবে এটি সবার জন্য একটি বড় সমস্যা সৃষ্টি করবে।তিনি বাড়ির চারপাশে তাকালেন এবং দরজাটি বের করার আগে তার চাবিগুলি এবং তার মানিব্যাগটি পেয়েছিলেন।\n\nমারিয়া যখন স্টোরের সামনের দিকে টানল তখন সে দেখে হতবাক হয়ে গেল যে লাইটগুলি চালু আছে তবে কেউ ভবনে নেই।তিনি নির্জন আইলগুলির মধ্য দিয়ে সাবধানে হাঁটলেন, সবাই কোথায় গিয়েছিল সে সম্পর্কে ক্লু খুঁজছিলেন।হঠাৎ করেই, তিনি তার পিছনে আন্দোলন শুনেছিলেন।তিনি দ্রুত ঘুরে দাঁড়ালেন, লড়াইয়ের জন্য প্রস্তুত।\n\nতবে তার পিছনে কোনও ব্যক্তিকে খুঁজে পাওয়ার পরিবর্তে তিনি দেখলেন সিরিয়াল আইলের মাঝখানে একটি বড় দুধ গরু দাঁড়িয়ে আছে।গ

In [31]:
reason_test_dataset[100]

{'conversations': [{'content': 'প্রশ্ন: Tidal Volume এর মান কত?\n\nঅপশন:\nA. 500 ml\nB. 400 mL\nC. 200 mL\nD. 300 mL',
   'role': 'user'},
  {'content': '<think>আচ্ছা, দেখা যাক। প্রশ্নটি হলো Tidal Volume এবং এর মান সম্পর্কে। আমার মনে আছে যে tidal volume হলো স্বাভাবিক শ্বাস-প্রশ্বাসের সময় ফুসফুসের ভিতরে এবং বাইরে যাওয়া বাতাসের পরিমাণ। কিন্তু অপেক্ষা করুন, সঠিক মানটি কত?\n\nহুমম, আমার মনে হয় শরীরবিদ্যায়, একজন সুস্থ প্রাপ্তবয়স্কের জন্য গড় tidal volume প্রায় 500 মিলিলিটার। তবে কখনও কখনও লোকেরা এটিকে অন্যান্য ভলিউমের সাথে বিভ্রান্ত করতে পারে যেমন inspiratory reserve volume বা expiratory reserve volume। আমাকে নিশ্চিত হতে দিন।\n\nঅপেক্ষা করুন, অপশনগুলো হলো A:500 ml, B:400, C:200, D:300। সুতরাং স্ট্যান্ডার্ড উত্তরটি 500 ml হওয়া উচিত। তবে এখানে কোনও কৌশল আছে কি? যেমন, বিশ্রামের সময় নাকি ব্যায়ামের সময় এটি আলাদা? না, tidal volume বিশেষভাবে স্বাভাবিক, বিশ্রামের শ্বাস-প্রশ্বাসের সময় হয়ে থাকে। সুতরাং হ্যাঁ, 500 ml। সুতরাং সঠিক উত্তরটি A হওয়া উচিত।</think>\n\nTidal Volume হলো স্বাভাবিক শ্বা

## **🦥 Train and Validation Dataset Preparation**

In [32]:
nonreason_train_dataset = nonreason_train_dataset.map(formatting_prompts_func, batched = True)
nonreason_test_dataset = nonreason_test_dataset.map(formatting_prompts_func, batched = True)
reason_train_dataset = reason_train_dataset.map(formatting_prompts_func, batched = True)
reason_test_dataset = reason_test_dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/8705 [00:00<?, ? examples/s]

Map:   0%|          | 0/446 [00:00<?, ? examples/s]

Map:   0%|          | 0/17791 [00:00<?, ? examples/s]

Map:   0%|          | 0/2564 [00:00<?, ? examples/s]

In [33]:
nonreason_test_dataset[100]["text"]

'<start_of_turn>user\nআপনি যদি দুধের জন্য মুদি দোকানে ভ্রমণে যান এমন কোনও মহিলার সম্পর্কে একটি ছোট গল্প লিখেন তবে আমি এটি চাই।দয়া করে 200 টিরও কম শব্দ তৈরি করুন এবং শেষে একটি চমক তৈরি করুন।<end_of_turn>\n<start_of_turn>model\nমারিয়া রেফ্রিজারেটরে তাকিয়ে দীর্ঘশ্বাস ফেলল যখন সে দেখল যে পরিবারটি আবার দুধের বাইরে চলে গেছে।তিনি জানতেন যে তার বাচ্চাদের সকালে প্রাতঃরাশের জন্য দুধের প্রয়োজন ছিল এবং যদি তিনি এখনই দোকানে না যান তবে এটি সবার জন্য একটি বড় সমস্যা সৃষ্টি করবে।তিনি বাড়ির চারপাশে তাকালেন এবং দরজাটি বের করার আগে তার চাবিগুলি এবং তার মানিব্যাগটি পেয়েছিলেন।\n\nমারিয়া যখন স্টোরের সামনের দিকে টানল তখন সে দেখে হতবাক হয়ে গেল যে লাইটগুলি চালু আছে তবে কেউ ভবনে নেই।তিনি নির্জন আইলগুলির মধ্য দিয়ে সাবধানে হাঁটলেন, সবাই কোথায় গিয়েছিল সে সম্পর্কে ক্লু খুঁজছিলেন।হঠাৎ করেই, তিনি তার পিছনে আন্দোলন শুনেছিলেন।তিনি দ্রুত ঘুরে দাঁড়ালেন, লড়াইয়ের জন্য প্রস্তুত।\n\nতবে তার পিছনে কোনও ব্যক্তিকে খুঁজে পাওয়ার পরিবর্তে তিনি দেখলেন সিরিয়াল আইলের মাঝখানে একটি বড় দুধ গরু দাঁড়িয়ে আছে।গরু জোরে জোর

In [34]:
reason_test_dataset[100]["text"]

'<start_of_turn>user\nপ্রশ্ন: Tidal Volume এর মান কত?\n\nঅপশন:\nA. 500 ml\nB. 400 mL\nC. 200 mL\nD. 300 mL<end_of_turn>\n<start_of_turn>model\n<think>আচ্ছা, দেখা যাক। প্রশ্নটি হলো Tidal Volume এবং এর মান সম্পর্কে। আমার মনে আছে যে tidal volume হলো স্বাভাবিক শ্বাস-প্রশ্বাসের সময় ফুসফুসের ভিতরে এবং বাইরে যাওয়া বাতাসের পরিমাণ। কিন্তু অপেক্ষা করুন, সঠিক মানটি কত?\n\nহুমম, আমার মনে হয় শরীরবিদ্যায়, একজন সুস্থ প্রাপ্তবয়স্কের জন্য গড় tidal volume প্রায় 500 মিলিলিটার। তবে কখনও কখনও লোকেরা এটিকে অন্যান্য ভলিউমের সাথে বিভ্রান্ত করতে পারে যেমন inspiratory reserve volume বা expiratory reserve volume। আমাকে নিশ্চিত হতে দিন।\n\nঅপেক্ষা করুন, অপশনগুলো হলো A:500 ml, B:400, C:200, D:300। সুতরাং স্ট্যান্ডার্ড উত্তরটি 500 ml হওয়া উচিত। তবে এখানে কোনও কৌশল আছে কি? যেমন, বিশ্রামের সময় নাকি ব্যায়ামের সময় এটি আলাদা? না, tidal volume বিশেষভাবে স্বাভাবিক, বিশ্রামের শ্বাস-প্রশ্বাসের সময় হয়ে থাকে। সুতরাং হ্যাঁ, 500 ml। সুতরাং সঠিক উত্তরটি A হওয়া উচিত।</think>\n\nTidal Volume হলো স্বাভাবিক শ্বাস-প্রশ্বাসে

## **🦥 Merge Train and Val Dataset**

In [35]:
from datasets import concatenate_datasets

In [36]:
train_data = concatenate_datasets([
    nonreason_train_dataset,
    reason_train_dataset
])


In [37]:

val_data = concatenate_datasets([
    nonreason_test_dataset,
    reason_test_dataset
])


In [38]:
print("Reason Train Data Length = ",len(reason_train_dataset))
print("Reason Test Data Length = ",len(reason_test_dataset))
print("Non-Reason Train Data Length = ",len(nonreason_train_dataset))
print("Non-Reason Test Data Length = ",len(nonreason_test_dataset))

print("\n================================\n")

print("Train Data Length = ",len(train_data))
print("Val Data Length = ",len(val_data))


Reason Train Data Length =  17791
Reason Test Data Length =  2564
Non-Reason Train Data Length =  8705
Non-Reason Test Data Length =  446


Train Data Length =  26496
Val Data Length =  3010


## **🦥 Training**

In [39]:
from trl import SFTTrainer, SFTConfig

In [40]:
sftconfig = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 8, # Effective batch size = 2 * 4 = 8
        warmup_steps = 5,
        max_steps = 10,                 # Short run for demonstration; set to None for full epochs
        # num_train_epochs = 1,         # Alternatively, train for n full epoch
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(), # Use bf16 if available, else fp16
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",           # Use 8-bit AdamW optimizer
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        per_device_eval_batch_size=8,
        seed = 42,
        dataloader_pin_memory=True, #fast gpu data transfer
        output_dir = "outputs",         # Directory to save checkpoints
        report_to = "none",             # Disable external reporting
        eval_strategy="steps",  # Evaluate during training
        eval_steps=10,                 # Evaluate every n steps
        fp16_full_eval = True,
        eval_accumulation_steps=1,
        load_best_model_at_end=True, # Load best model based on evaluation metric
        metric_for_best_model="eval_loss",  
        greater_is_better=False,           # For accuracy, higher is better
        dataset_num_proc=1
    )

In [41]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    eval_dataset = val_data, # Can set up evaluation!
    args = sftconfig
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=1):   0%|          | 0/26496 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=1):   0%|          | 0/3010 [00:00<?, ? examples/s]

In [42]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=8):   0%|          | 0/26496 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/3010 [00:00<?, ? examples/s]

In [43]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

"<bos><start_of_turn>user\nআমি বাচ্চা করছি, এবং আমার কারুশিল্পের কমপক্ষে 3 টি ধারণা প্রয়োজন।উপকরণগুলি অবশ্যই আমি ইয়ার্ডে খুঁজে পেতে পারি পরিবারের উপকরণ বা স্টাফ হতে হবে।জিনিসগুলিকে একসাথে রাখার জন্য আমার কাছে কেবল আঠালো এবং টেপ রয়েছে।আমার সাজসজ্জার জন্য পেইন্ট আছে।<end_of_turn>\n<start_of_turn>model\nকোনও সমস্যা নেই, এখানে তিনটি ছাগলছানা-বান্ধব নৈপুণ্য আইডিয়া রয়েছে যা আপনি বাড়ির চারপাশে আইটেমগুলি দিয়ে তৈরি করতে পারেন:\n\n1. আঁকা শিলা: এই নৈপুণ্য সবই সৃজনশীলতা সম্পর্কে!বাচ্চারা যা চায় তা আঁকতে পারে বা আপনি কোনও থিমের সিদ্ধান্ত নিতে পারেন এবং তার উপর ভিত্তি করে শিলাগুলি আঁকতে পারেন।\n২. পেপার টিউব প্রাণী: আপনি এটির জন্য খালি টয়লেট পেপার বা কাগজের তোয়ালে টিউব ব্যবহার করতে পারেন।অবজেক্টটি হ'ল আপনার টিউবটিকে একটি ছোট প্রাণীর মতো দেখায়;আপনি উপরে কান কেটে ফেলতে পারেন এবং চেহারাটি সম্পূর্ণ করতে ছোট মুখ, পা ইত্যাদি আঁকতে পারেন।\n৩. ম্যাকারনি গহনা: কিছু পাস্তা নুডলস এবং স্ট্রিং দিয়ে আপনার নিজস্ব ফ্যাশন লাইন তৈরি করুন!নিশ্চিত হয়ে নিন যে আপনি নুডলস ব্যবহার করেছেন যা তাদের মাধ্যমে স্ট্

In [44]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

"                                                                         কোনও সমস্যা নেই, এখানে তিনটি ছাগলছানা-বান্ধব নৈপুণ্য আইডিয়া রয়েছে যা আপনি বাড়ির চারপাশে আইটেমগুলি দিয়ে তৈরি করতে পারেন:\n\n1. আঁকা শিলা: এই নৈপুণ্য সবই সৃজনশীলতা সম্পর্কে!বাচ্চারা যা চায় তা আঁকতে পারে বা আপনি কোনও থিমের সিদ্ধান্ত নিতে পারেন এবং তার উপর ভিত্তি করে শিলাগুলি আঁকতে পারেন।\n২. পেপার টিউব প্রাণী: আপনি এটির জন্য খালি টয়লেট পেপার বা কাগজের তোয়ালে টিউব ব্যবহার করতে পারেন।অবজেক্টটি হ'ল আপনার টিউবটিকে একটি ছোট প্রাণীর মতো দেখায়;আপনি উপরে কান কেটে ফেলতে পারেন এবং চেহারাটি সম্পূর্ণ করতে ছোট মুখ, পা ইত্যাদি আঁকতে পারেন।\n৩. ম্যাকারনি গহনা: কিছু পাস্তা নুডলস এবং স্ট্রিং দিয়ে আপনার নিজস্ব ফ্যাশন লাইন তৈরি করুন!নিশ্চিত হয়ে নিন যে আপনি নুডলস ব্যবহার করেছেন যা তাদের মাধ্যমে স্ট্রিং থ্রেডযুক্ত থাকতে পারে এবং আপনার স্টাইলের সাথে ফিট করার জন্য নুডলগুলি আঁকতে পারে!<end_of_turn>\n"

In [45]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 26,496 | Num Epochs = 1 | Total steps = 10
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 8 x 1) = 64
 "-____-"     Trainable parameters = 13,045,760 of 1,012,931,712 (1.29% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
10,2.469800,2.588514


Unsloth: Not an error, but Gemma3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


## **🦥 Streaming Inference**

In [46]:
from transformers import TextStreamer

In [47]:

query = "আমি কি খাই তা দেখার বিষয়ে আমাকে দয়া করে একটি কবিতা দিতে পারেন?এর নাম দিন 'আপনি স্বাস্থ্যকর'।"

messages = [
    {"role" : "user", "content" :  query}
]

In [48]:
# Format the prompt, explicitly DISABLING thinking mode
text_input_no_think = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Crucial for generation
    enable_thinking = False,      # *** Disable thinking ***
)


print("\n--- Non-Thinking Inference ---\n")
print("Formatted Input:\n", text_input_no_think)
print("\n--- Non-Thinking Inference ---\n")


# Generate response using parameters suitable for non-thinking/chat
print("\n-----------------------------\n")
inputs = tokenizer(text_input_no_think, return_tensors = "pt").to("cuda")
streamer_no_think = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    **inputs,
    max_new_tokens = 100,
    temperature = 0.1, # Recommended for chat
    top_p = 0.9,       # Recommended for chat
    top_k = 64,
    repetition_penalty=1.15,
    streamer = streamer_no_think,
    eos_token_id = tokenizer.eos_token_id # Ensure generation stops properly
)
print("\n-----------------------------\n")


--- Non-Thinking Inference ---

Formatted Input:
 <bos><start_of_turn>user
আমি কি খাই তা দেখার বিষয়ে আমাকে দয়া করে একটি কবিতা দিতে পারেন?এর নাম দিন 'আপনি স্বাস্থ্যকর'।<end_of_turn>
<start_of_turn>model


--- Non-Thinking Inference ---


-----------------------------

অবশ্যই, এখানে "আপনি স্বাস্থ্যকর" নামক একটি কবিতা রয়েছে:

আপনার শরীরটি আপনার শরীরের সমস্ত শক্তি এবং প্রোটিনের সাথে সম্পর্কিত।
আপনার খাদ্য গ্রহণ করা উচিত না হয় না, আপনি যদি চান তবে এটি আপনাকে আরও ভাল দেখতে সাহায্য করবে।
আপনি যে খাবারগুলি বেছে নিতে পারেন তার কিছু টিপস নিচে দেওয়া হল:
- প্রচুর পরিমাণে ফল এবং সবজি খান।
- শস্য বা অন্যান্য শস্যের মতো খাবার পরিহার করুন।
- মিষ্টি পানীয় থেকে

-----------------------------

